In [1]:
from sentence_transformers import SentenceTransformer

model_name = 'multi-qa-distilbert-cos-v1'
model = SentenceTransformer(model_name)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.52k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/523 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Q1 - What's the first value of the resulting vector?
# 0.07

In [2]:
user_question = "I just discovered the course. Can I still join it?"

In [6]:
v = model.encode(user_question)

In [7]:
v, len(v)

(array([ 7.82226101e-02, -4.01311480e-02,  3.86136100e-02, -1.78964794e-04,
         8.92346725e-02, -5.04590981e-02, -1.05026886e-02,  3.71056087e-02,
        -4.18713987e-02,  3.48085053e-02, -1.20702377e-02, -2.36942451e-02,
         3.87900136e-02,  1.60988253e-02,  3.50747332e-02,  3.04753659e-03,
         5.79672046e-02, -4.10627723e-02, -3.41552719e-02, -2.56396439e-02,
        -3.55264023e-02,  1.42908022e-02, -1.62800048e-02,  3.21446285e-02,
        -4.66897413e-02,  7.89185986e-02,  4.90160994e-02,  1.56760979e-02,
        -1.69110280e-02,  2.26482172e-02,  5.60206063e-02, -3.98361683e-02,
         6.77409545e-02, -1.20210098e-02,  1.12624257e-03, -1.94394719e-02,
        -2.65951324e-02,  1.06178010e-02,  1.69687588e-02,  1.13487877e-02,
        -2.97063254e-02,  5.25258556e-02, -1.41453082e-02,  4.61699590e-02,
         1.17066093e-02, -2.38052942e-02, -6.32557943e-02, -1.92042012e-02,
        -7.10596424e-03,  3.24167609e-02,  2.49618050e-02, -5.27497754e-03,
         2.0

In [ ]:
# Q2 - What's the shape of X? (X.shape). Include the parantheses.

# (375, 768)

In [9]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [13]:
mlops_documents = []
for doc in documents:
    if doc['course'] == 'machine-learning-zoomcamp':
        mlops_documents.append(doc)

len(mlops_documents)

375

In [12]:
from tqdm.auto import tqdm

In [15]:
for doc in tqdm(mlops_documents):
    question = doc['question']
    text = doc['text']
    qa_text = f'{question} {text}'

    doc['question_vector'] = model.encode(question)
    doc['text_vector'] = model.encode(text)
    doc['question_text_vector'] = model.encode(qa_text)

  0%|          | 0/375 [00:00<?, ?it/s]

In [17]:
len(doc['question_text_vector'])

768

In [43]:
doc

{'text': 'I’ve seen LinkedIn users list DataTalksClub as Experience with titles as:\nMachine Learning Fellow\nMachine Learning Student\nMachine Learning Participant\nMachine Learning Trainee\nPlease note it is best advised that you do not list the experience as an official “job” or “internship” experience since DataTalksClub did not hire you, nor financially compensate you.\nOther ways you can incorporate the experience in the following sections:\nOrganizations\nProjects\nSkills\nFeatured\nOriginal posts\nCertifications\nCourses\nBy Annaliese Bronz\nInteresting question, I put the link of my project into my CV as showcase and make posts to show my progress.\nBy Ani Mkrtumyan',
 'section': 'Miscellaneous',
 'question': 'Any advice for adding the Machine Learning Zoomcamp experience to your LinkedIn profile?',
 'course': 'machine-learning-zoomcamp',
 'id': 'c6a22665',
 'question_vector': array([ 5.49459197e-02, -3.79322916e-02,  6.19464070e-02,  6.17474988e-02,
         1.04033791e-01, -

In [18]:
embeddings = []
for doc in tqdm(mlops_documents):
    question = doc['question']
    text = doc['text']
    qa_text = f'{question} {text}'

    embeddings.append(model.encode(qa_text))

  0%|          | 0/375 [00:00<?, ?it/s]

In [19]:
import numpy as np 

X = np.array(embeddings) 
X.shape

(375, 768)

In [23]:
"""
We have the embeddings and the query vector. 
Now let's compute the cosine similarity between the vector from Q1 (let's call it v) and the matrix from Q2.

The vectors returned from the embedding model are already normalized 
(you can check it by computing a dot product of a vector with itself - it should return something very close to 1.0). 
This means that in order to compute the coside similarity, it's sufficient to multiply the matrix X by the vector v:
"""

# 0.65

"\nWe have the embeddings and the query vector. \nNow let's compute the cosine similarity between the vector from Q1 (let's call it v) and the matrix from Q2.\n\nThe vectors returned from the embedding model are already normalized \n(you can check it by computing a dot product of a vector with itself - it should return something very close to 1.0). \nThis means that in order to compute the coside similarity, it's sufficient to multiply the matrix X by the vector v:\n"

In [22]:
scores = X.dot(v)

max(scores)

0.6506575

In [ ]:
# Q4 - Evaulate our search engine 

# Hit-rate for our search engine: 0.93

In [25]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=mlops_documents, embeddings=X)
search_engine.search(v, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693',
  'question_vector': array([ 0.07717118, -0.04749377,  0.02866323, -0.01141068,  0.08245005,
         -0.04042275, -0.02613377,  0.04122073, -0.04840765,  0.01509398,
         -0.0014967 , -0.01334827,  0.0461828 ,  0.02318393,  0.04547327,
         -0.00809984,  0.07718321, -0.03334848, -0.04180019, -0.02304639,
         -0.01866887,  0.00298914, -0.0063176 ,  0.039316  , -0.0228994 ,
          0.07724466,  0.06296353,  0.037800

In [29]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [34]:
ground_truth[0]

{'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp',
 'document': '0227b872'}

In [69]:
def evaluate(ground_truth, search_function):
    relevance_total = []
    # store the search result to the relevant (each will have knn=5 relevant docs)
    # then calculate the metrics through all the docs using hit rate and mrr

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        # 'mrr': mrr(relevance_total),
    }

In [68]:
# put into function
def question_text_vector_search_engine(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return search_engine.search(v_q, num_results=5)

In [70]:
evaluate(ground_truth, question_text_vector_search_engine)

  0%|          | 0/1830 [00:00<?, ?it/s]

{'hit_rate': 0.9398907103825137}

In [45]:
# Evaluate the search engine
relevance_total = []
    # store the search result to the relevant (each will have knn=5 relevant docs)
    # then calculate the metrics through all the docs using hit rate and mrr

for q in tqdm(ground_truth):
    doc_id = q['document']
    q = model.encode(q['question'])
    results = search_engine.search(q, num_results=5)
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

  0%|          | 0/1830 [00:00<?, ?it/s]

In [46]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [47]:
hit_rate(relevance_total)

0.9398907103825137

In [40]:
# Q5 - Indexing with Elasticsearch

# ee58a693

In [51]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
            "text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [62]:
def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        source = hit['_source']
        source['score'] = hit['_score']  # Add the score to the document
        result_docs.append(hit['_source'])

    return result_docs

In [58]:
for doc in tqdm(mlops_documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/375 [00:00<?, ?it/s]

In [63]:
elastic_search_knn('question_text_vector', v, 'machine-learning-zoomcamp')

[{'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'id': 'ee58a693',
  'score': 0.82532895},
 {'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It mea

In [67]:
# Q6 -  Hit-rate for Elasticsearch

# 0.91

In [ ]:
def evaluate(ground_truth, search_function):
    relevance_total = []
    # store the search result to the relevant (each will have knn=5 relevant docs)
    # then calculate the metrics through all the docs using hit rate and mrr

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [65]:
def question_text_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn('question_text_vector', v_q, course)

In [66]:
evaluate(ground_truth, question_text_vector_knn)

  0%|          | 0/1830 [00:00<?, ?it/s]

{'hit_rate': 0.9103825136612022}